In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib_venn import venn2

# Data

I performed an open ionbot and msfragger search (oxididation of M variable, max 2 missed cleavages).

See SOP document for more details on how MSFragger was ran.

In [23]:
#set this your local folder as the data was pushed to the repo as well
folder = "/home/compomics/tools/ionbot_paper"
datasets = ["PXD001468","PXD000561","PXD000815","PXD001250","TMTCPTAC"]
#datasets = ["PXD000561"]

 # Results
 
Load ionbot PSM results (first-matches + coeluting matches + features) + protein results.

In [26]:
def contains_missed_cleavage(x):
    if "K" in x[:-1]:
        return True
    if "R" in x[:-1]:
        return True
    return False

def is_in_s(x):
    return x in s

def get_scan_msfragger(x):
    return int(x.split('.')[-2])
def get_spectrum_file_msfragger(x):
    return x.split('.')[0]

def replace_L_I(x):
    return x.replace("L","I")

def is_decoy(x):
    return "decoy_" in x

def get_universal_link(x):
    if str(x["modifications"]) == "nan":
        return "http://proteomecentral.proteomexchange.org/usi/?usi=mzspec:%s:%s:scan:%i:%s/%i"%(x["dataset"],x["sfile"],x["scan"],x["matched_peptide"],x["charge"])
    tmp = x["modifications"].split("|")
    seq = list(x["matched_peptide"])
    for i in range(0,len(tmp),2):
        mod = int(tmp[i+1].split(']')[0][1:])
        if mod == 4:
            continue
        pos = int(tmp[i])
        seq.insert(pos,"[UNIMOD:%i]"%mod)
    return "http://proteomecentral.proteomexchange.org/usi/?usi=mzspec:%s:%s:scan:%i:%s/%i"%(x["dataset"],x["sfile"],x["scan"],''.join(seq),x["charge"])
    

In [29]:
fi = 1
for i,dataset in enumerate(datasets):
    print(dataset)
    #Read ionbot results
    if "PXD001468" in dataset:
        def get_scan_ionbot(x):
            return int(x.split('.')[1])
        def get_spectrum_file_ionbot(x):
            return x.split('.')[0]
    else:
        def get_scan_ionbot(x):
            print(x)
            return int(x.split('=')[-1])
        def get_spectrum_file_ionbot(x):
            return x.split('.')[0]
    folder2 = folder + "/" + dataset
    ionbot = pd.read_csv("%s/ionbot.first.csv"%folder2)
    ionbot["rank"] = ["first"]*len(ionbot)
    tmp = pd.read_csv("%s/ionbot.lower.csv"%folder2)
    tmp["rank"] = ["lower"]*len(tmp)
    ionbot = pd.concat([ionbot,tmp])
    tmp = pd.read_csv("%s/ionbot.features.csv"%folder2)
    ionbot = ionbot.merge(tmp,on="ionbot_match_id",how="left")
    ionbot = ionbot[(ionbot["database"]=="T")&(ionbot["q-value"]<=0.01)]
    if "PXD001468" in dataset:
        ionbot["scan"] = ionbot["spectrum_title"].apply(get_scan_ionbot)
        ionbot["sfile"] = ionbot["spectrum_title"].apply(get_spectrum_file_ionbot)
    else:
        ionbot["sfile"] = ionbot["spectrum_file"].apply(get_spectrum_file_ionbot)
    ionbot["num_passed"] = ionbot.groupby(["sfile","scan"])["scan"].transform("count")
    ionbot["PSM"] = ionbot["scan"].astype(str) + ionbot["sfile"] + ionbot["database_peptide"].apply(replace_L_I)

    #Read MSfragger (Prophet) result
    fragger = pd.read_csv("%s/msfragger/%s/noprotein/psm.tsv"%(folder,dataset),sep="\t")
    fragger["scan"] = fragger["Spectrum"].apply(get_scan_msfragger)
    #sfile is used to filter on spectrum file
    fragger["sfile"] = fragger["Spectrum"].apply(get_spectrum_file_msfragger)
    fragger["PSM"] = fragger["scan"].astype(str) + fragger["sfile"] + fragger["Peptide"].apply(replace_L_I)

    #adding MS2PIP correlation and retention time prediction error features:
    data_folder = "/home/compomics/extra_disk/data/%s/"%dataset
    sfiles = fragger["sfile"].unique()
    X = []
    for sf in sfiles:
        tmp = pd.read_csv("%s/%s_fragger.peprec_ionbotpeprec/ionbot.csv"%(data_folder,sf))
        tmp["sfile"] = [sf]*len(tmp)
        X.append(tmp)
    fragger_stats = pd.concat(X)
    if dataset == "PXD001468":
        def get_scan(x):
            return int(x.split(".")[1])
        fragger_stats['scan'] = fragger_stats["title"].apply(get_scan)
    fragger_stats["by-count"] = fragger_stats["count_b"] + fragger_stats["count_y"]
    fragger_stats["by-explained"] = fragger_stats["explained_b"] + fragger_stats["explained_y"]
    tmp = fragger_stats[["scan","sfile","corr","rt_diff_A","by-count","by-explained"]]
    fragger = fragger.merge(tmp,on=["scan","sfile"],how="left")

    #write file with co-eluting matches
    tmp = ionbot[ionbot["num_passed"]>1][["sfile","spectrum_title","scan","rank","matched_peptide","modifications","charge","by-intensity-pattern-correlation","rt-pred-error","proteins"]]
    tmp["dataset"] = [dataset]*len(tmp)
    tmp["USI"] = tmp.apply(get_universal_link,axis=1)
    tmp.to_csv("%s_coeluting_matches.csv"%dataset,index=False)
    print(len(tmp))
    
    #write matches unique to msfragger
    s1 = set(ionbot[ionbot["rank"]=="first"]["PSM"])
    s2 = set(fragger["PSM"])

    s = s2.intersection(s1)
    fragger["unique"] = fragger["PSM"].apply(is_in_s)
    print(len(fragger))
    print(fragger.columns)
    tmp = fragger[fragger["unique"]==True][["sfile","Spectrum","scan","Peptide","Assigned Modifications","Charge","corr","rt_diff_A","Protein"]]
    tmp.columns = [["sfile","spectrum_title","scan","matched_peptide","modifications","charge","by-intensity-pattern-correlation","rt-pred-error","proteins"]]
    dd
    

PXD001468
54789
772186
Index(['Spectrum', 'Spectrum File', 'Peptide', 'Modified Peptide', 'Prev AA',
       'Next AA', 'Peptide Length', 'Charge', 'Retention', 'Observed Mass',
       'Calibrated Observed Mass', 'Observed M/Z', 'Calibrated Observed M/Z',
       'Calculated Peptide Mass', 'Calculated M/Z', 'Delta Mass',
       'Expectation', 'Hyperscore', 'Nextscore', 'PeptideProphet Probability',
       'Number of Enzymatic Termini', 'Number of Missed Cleavages',
       'Protein Start', 'Protein End', 'Intensity', 'Assigned Modifications',
       'Observed Modifications', 'MSFragger Localization',
       'Best Score with Delta Mass', 'Best Score without Delta Mass',
       'Is Unique', 'Protein', 'Protein ID', 'Entry Name', 'Gene',
       'Protein Description', 'Mapped Genes', 'Mapped Proteins', 'scan',
       'sfile', 'PSM', 'corr', 'rt_diff_A', 'by-count', 'by-explained',
       'unique'],
      dtype='object')


NameError: name 'dd' is not defined